# Predict Next Purchase

In this tutorial, build a machine learning application that predicts whether customers will purchase a product within the next shopping period. This application is structured into three important steps:

- Prediction Engineering
- Feature Engineering
- Machine Learning

In the first step, you generate new labels from the data by using [Compose](https://compose.alteryx.com/). In the second step, you generate features for the labels by using [Featuretools](https://featuretools.alteryx.com/). In the third step, you search for the best machine learning pipeline by using [EvalML](https://evalml.alteryx.com/). After working through these steps, you should understand how to build machine learning applications for real-world problems like predicting consumer spending.

**Note: In order to run this example, you should have Featuretools 1.4.0 or newer and EvalML 0.41.0 or newer installed.**

In [ ]:
from demo.next_purchase import load_sample
from matplotlib.pyplot import subplots
import composeml as cp
import pandas as pd
import featuretools as ft
import evalml

Use this historical data of online grocery orders provided by [Instacart](https://www.kaggle.com/c/instacart-market-basket-analysis/data).

In [ ]:
df = load_sample()
df.head()

## Prediction Engineering

> Will customers purchase a product within the next shopping period?

In this prediction problem, there are two parameters:

- The product that a customer can purchase.
- The length of the shopping period.

You can change these parameters to create different prediction problems. For example, will a customer purchase a banana within the next 3 days or an avocado within the next three weeks? These variations can be done by simply tweaking the parameters. This helps you explore different scenarios that are crucial for making better decisions.

### Defining the Labeling Function

Start by defining a labeling function that checks if a customer bought a given product. Make the product a parameter of the function. Our labeling function is used by a label maker to extract the training examples.

In [ ]:
def bought_product(ds, product_name):
    return ds.product_name.str.contains(product_name).any()

### Representing the Prediction Problem

Represent the prediction problem by creating a label maker with the following parameters:

- `target_dataframe_index` as the columns for the customer ID, since you want to process orders for each customer.
- `labeling_function` as the function you defined previously.
- `time_index` as the column for the order time. The shoppings periods are based on this time index.
- `window_size` as the length of a shopping period. You can easily change this parameter to create variations of the prediction problem.

In [ ]:
lm = cp.LabelMaker(
    target_dataframe_index="user_id",
    time_index="order_time",
    labeling_function=bought_product,
    window_size="3d",
)

### Finding the Training Examples

Run a search to get the training examples by using the following parameters:

- The grocery orders sorted by the order time, since the search expects the orders to be sorted chronologically. Otherwise, an error is raised.
- `num_examples_per_instance` to find the number of training examples per customer. In this case, the search returns all existing examples.
- `product_name` as the product to check for purchases. This parameter gets passed directly to the our labeling function.
- `minimum_data` as the amount of data that is used to make features for the first training example.

In [ ]:
lt = lm.search(
    df.sort_values("order_time"),
    num_examples_per_instance=-1,
    product_name="Banana",
    minimum_data=pd.Timedelta("3d"),
    verbose=False,
)

lt.head()

The output from the search is a label times table with three columns:

- The customer ID associated to the orders. There can be many training examples generated from each customer.
- The start time of the shopping period. This is also the cutoff time for building features. Only data that existed beforehand is valid to use for predictions.
- Whether the product was purchased during the shopping period window. This is calculated by our labeling function.

As a helpful reference, you can print out the search settings that were used to generate these labels. The description also shows us the label distribution which we can check for imbalanced labels.

In [ ]:
lt.describe()

You can get a better look at the labels by plotting the distribution and cumulative count across time.

In [ ]:
%matplotlib inline
fig, ax = subplots(nrows=2, ncols=1, figsize=(6, 8))
lt.plot.distribution(ax=ax[0])
lt.plot.count_by_time(ax=ax[1])
fig.tight_layout(pad=2)

## Feature Engineering

In the previous step, you generated the labels. The next step is to generate features.

### Representing the Data

Start by representing the data with an EntitySet. That way, you can generate features based on the relational structure of the dataset. You currently have a single table of orders where one customer can have many orders. This one-to-many relationship can be represented by normalizing a customer dataframe. The same can be done for other one-to-many relationships like aisle-to-products. Because you want to make predictions based on the customer, you should use this customer dataframe as the target for generating features.

In [ ]:
es = ft.EntitySet("instacart")

es.add_dataframe(
    dataframe=df.reset_index(),
    dataframe_name="order_products",
    time_index="order_time",
    index="id",
)

es.normalize_dataframe(
    base_dataframe_name="order_products",
    new_dataframe_name="orders",
    index="order_id",
    additional_columns=["user_id"],
    make_time_index=False,
)

es.normalize_dataframe(
    base_dataframe_name="orders",
    new_dataframe_name="customers",
    index="user_id",
    make_time_index=False,
)

es.normalize_dataframe(
    base_dataframe_name="order_products",
    new_dataframe_name="products",
    index="product_id",
    additional_columns=["aisle_id", "department_id"],
    make_time_index=False,
)

es.normalize_dataframe(
    base_dataframe_name="products",
    new_dataframe_name="aisles",
    index="aisle_id",
    additional_columns=["department_id"],
    make_time_index=False,
)

es.normalize_dataframe(
    base_dataframe_name="aisles",
    new_dataframe_name="departments",
    index="department_id",
    make_time_index=False,
)

es.add_interesting_values(
    dataframe_name="order_products",
    values={"department": ["produce"], "product_name": ["Banana"]},
)
es.plot()

### Calculating the Features

Now you can generate features by using a method called Deep Feature Synthesis (DFS). That method automatically builds features by stacking and applying mathematical operations called primitives across relationships in an entityset. The more structured an entityset is, the better DFS can leverage the relationships to generate better features. Let’s run DFS using the following parameters:

- `entity_set` as the entityset we structured previously.
- `target_dataframe_name` as the customer dataframe.
- `cutoff_time` as the label times that we generated previously. The label values are appended to the feature matrix.

In [ ]:
fm, fd = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    cutoff_time=lt,
    cutoff_time_in_index=True,
    include_cutoff_time=False,
    verbose=False,
)

fm.head()

There are two outputs from DFS: a feature matrix and feature definitions. The feature matrix is a table that contains the feature values with the corresponding labels based on the cutoff times. Feature definitions are features in a list that can be stored and reused later to calculate the same set of features on future data.

## Machine Learning

In the previous steps, you generated the labels and features. The final step is to build the machine learning pipeline.

### Splitting the Data

Start by extracting the labels from the feature matrix and splitting the data into a training set and a holdout set.

In [ ]:
fm.reset_index(drop=True, inplace=True)
y = fm.ww.pop("bought_product")

splits = evalml.preprocessing.split_data(
    X=fm,
    y=y,
    test_size=0.2,
    random_seed=0,
    problem_type="binary",
)

X_train, X_holdout, y_train, y_holdout = splits

### Finding the Best Model

Run a search on the training set to find the best machine learning model. During the search process, predictions from several different pipelines are evaluated.

In [ ]:
automl = evalml.AutoMLSearch(
    X_train=fm,
    y_train=y,
    problem_type="binary",
    objective="f1",
    random_seed=0,
    allowed_model_families=["catboost", "random_forest"],
    max_iterations=3,
)

automl.search()

Once the search is complete, you can print out information about the best pipeline found, like the parameters in each component.

In [ ]:
automl.best_pipeline.describe()
automl.best_pipeline.graph()

Score the model performance by evaluating predictions on the holdout set.

In [ ]:
best_pipeline = automl.best_pipeline.fit(X_train, y_train)

score = best_pipeline.score(
    X=X_holdout,
    y=y_holdout,
    objectives=["f1"],
)

dict(score)

From the pipeline, you can see which features are most important for predictions.

In [ ]:
feature_importance = best_pipeline.feature_importance
feature_importance = feature_importance.set_index("feature")["importance"]
top_k = feature_importance.abs().sort_values().tail(20).index
feature_importance[top_k].plot.barh(figsize=(8, 8), fontsize=14, width=0.7);

### Making Predictions

You are ready to make predictions with your trained model. Start by calculating the same set of features by using the feature definitions. Also, use a cutoff time based on the latest information available in the dataset.

In [ ]:
fm = ft.calculate_feature_matrix(
    features=fd,
    entityset=es,
    cutoff_time=ft.pd.Timestamp("2015-03-02"),
    cutoff_time_in_index=True,
    verbose=False,
)

fm.head()

Predict whether customers will purchase bananas within the next 3 days.

In [ ]:
y_pred = best_pipeline.predict(fm)
y_pred = y_pred.values

prediction = fm[[]]
prediction["bought_product (estimate)"] = y_pred
prediction.head()

### Next Steps

You have completed this tutorial. You can revisit each step to explore and fine-tune the model using different parameters until it is ready for production. For more information about how to work with the features produced by Featuretools, take a look at [the Featuretools documentation](https://featuretools.alteryx.com/). For more information about how to work with the models produced by EvalML, take a look at [the EvalML documentation](https://evalml.alteryx.com/).